In [19]:
import pdb
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.contrib.layers as layers

In [ ]:
import simulation_data_generate as sdg

In [ ]:
# simple situation
# just one kind of cell

# x -- RNAseq, y -- ATACseq
# x1,y2 true data, x2,y1 false data
# x1 -> y1 G:generator, Dy:discriminator
# y2 -> x1 F:generator, Dx:discriminator

# input data
data = data

In [36]:
class scRAP(object):
    """
    a simple situation version
    """
    
    def __init__(m,n,learning_rate):
        self._m = m
        self._n = n
        self.learning_rate = learning_rate
        
    @staticmethod
    def lrelu(x,alpha=0.2):
        with tf.variable_scope('leakyRelu'):
            return tf.maximum(x,alpha*x)
    
    # generator G
    def G(self,Z,dim_Z):
        with tf.variable_scope("G"):
            
            dim_1 = 2*dim_z
            dim_2 = 2*self._n
            dim_3 = self._n
            
            W1 = tf.get_variable("G_W1",shape=[dim_Z,dim_1],
                                 dtype=tf.float32,initializer=tf.random_normal_initializer(stddev=0.1))
            W2 = tf.get_variable("G_W2",shape=[dim_1,dim_2],
                                 dtype=tf.float32,initializer=tf.random_normal_initializer(stddev=0.1))
            W3 = tf.get_variable("G_W3",shape=[dim_2,dim_3],
                                 dtype=tf.float32,initializer=tf.random_normal_initializer(stddev=0.1))
            B1 = tf.get_variable("G_B1",shape=[dim_1],dtype=tf.float32,initializer=tf.constant_initializer())
            B2 = tf.get_variable("G_B2",shape=[dim_2],dtype=tf.float32,initializer=tf.constant_initializer())
            B3 = tf.get_variable("G_B3",shape=[dim_3],dtype=tf.float32,initializer=tf.constant_initializer())
            
            fc1 = self.lrelu(tf.add(tf.matmul(Z,W1)+B1))
            fc2 = self.lrelu(tf.add(tf.matmul(fc1,W2)+B2))
            fc3 = tf.nn.sigmoid(tf.add(tf.matmul(fc2,W3)+B3))
            return fc3
        
    # generator F
    def F(self,Z,dim_Z):
        with tf.variable_scope("F"):
            
            dim_1 = 2*dim_Z
            dim_2 = 2*self._m
            dim_3 = self._m
            
            W1 = tf.get_variable("F_W1",shape=[dim_Z,dim_1],
                                 dtype=tf.float32,initializer=tf.random_normal_initializer(stddev=0.1))
            W2 = tf.get_variable("F_W2",shape=[dim_1,dim_2],
                                 dtype=tf.float32,initializer=tf.random_normal_initializer(stddev=0.1))
            W3 = tf.get_variable("F_W3",shape=[dim_2,dim_3],
                                 dtype=tf.float32,initializer=tf.random_normal_initializer(stddev=0.1))
            B1 = tf.get_variable("F_B1",shape=[dim_1],dtype=tf.float32,initializer=tf.constant_initializer())
            B2 = tf.get_variable("F_B2",shape=[dim_2],dtype=tf.float32,initializer=tf.constant_initializer())
            B3 = tf.get_variable("F_B3",shape=[dim_3],dtype=tf.float32,initializer=tf.constant_initializer())
            
            fc1 = self.lrelu(tf.add(tf.matmul(Z,W1)+B1))
            fc2 = tf.nn.sigmoid(tf.add(tf.matmul(fc1,W2)+B2))
            fc3 = tf.add(tf.matmul(fc2,W3)+B3)
            return fc3
    
    # discriminator Dy
    def Dy(self,Y):
        with tf.variable_scope("Dy"):
            
            dim_1 = self._n//2
            dim_2 = self._n//4
            dim_3 = 1
            
            W1 = tf.get_variable("Dy_W1",shape=[dim_Z,dim_1],
                                 dtype=tf.float32,initializer=tf.random_normal_initializer(stddev=0.1))
            W2 = tf.get_variable("Dy_W2",shape=[dim_1,dim_2],
                                 dtype=tf.float32,initializer=tf.random_normal_initializer(stddev=0.1))
            W3 = tf.get_variable("Dy_W3",shape=[dim_2,dim_3],
                                 dtype=tf.float32,initializer=tf.random_normal_initializer(stddev=0.1))
            B1 = tf.get_variable("Dy_B1",shape=[dim_1],dtype=tf.float32,initializer=tf.constant_initializer())
            B2 = tf.get_variable("Dy_B2",shape=[dim_2],dtype=tf.float32,initializer=tf.constant_initializer())
            B3 = tf.get_variable("Dy_B3",shape=[dim_3],dtype=tf.float32,initializer=tf.constant_initializer())
            
            fc1 = self.lrelu(tf.add(tf.matmul(Y,W1)+B1))
            fc2 = tf.nn.sigmoid(tf.add(tf.matmul(fc1,W2)+B2))
            fc3 = tf.add(tf.matmul(fc2,W3)+B3)
            return fc3,tf.nn.sigmoid(fc3)
        
     # discriminator Dx
    def Dx(self,X):
        with tf.variable_scope("Dy"):
            
            dim_1 = self._m//2
            dim_2 = self._m//4
            dim_3 = 1
            
            W1 = tf.get_variable("Dx_W1",shape=[dim_Z,dim_1],
                                 dtype=tf.float32,initializer=tf.random_normal_initializer(stddev=0.1))
            W2 = tf.get_variable("Dx_W2",shape=[dim_1,dim_2],
                                 dtype=tf.float32,initializer=tf.random_normal_initializer(stddev=0.1))
            W3 = tf.get_variable("Dx_W3",shape=[dim_2,dim_3],
                                 dtype=tf.float32,initializer=tf.random_normal_initializer(stddev=0.1))
            B1 = tf.get_variable("Dx_B1",shape=[dim_1],dtype=tf.float32,initializer=tf.constant_initializer())
            B2 = tf.get_variable("Dx_B2",shape=[dim_2],dtype=tf.float32,initializer=tf.constant_initializer())
            B3 = tf.get_variable("Dx_B3",shape=[dim_3],dtype=tf.float32,initializer=tf.constant_initializer())
            
            fc1 = self.lrelu(tf.add(tf.matmul(X,W1)+B1))
            fc2 = tf.nn.sigmoid(tf.add(tf.matmul(fc1,W2)+B2))
            fc3 = tf.add(tf.matmul(fc2,W3)+B3)
            return fc3,tf.nn.sigmoid(fc3)
        

In [37]:
def get_batch(data,batch_size):
    input_queue = tf.train.slice_input_producer([data], num_epochs=1, shuffle=True, capacity=32 ) 
    batch = tf.train.batch(input_queue, batch_size=batch_size, num_threads=1, capacity=32, allow_smaller_final_batch=False)
    return batch

In [39]:
def train(data,m,n,learning_rate,batch_size,lamda):
    data_x = data["x"]
    data_y = data["y"]
    func = scRAP(m,n)
    
    with tf.variable_scope("placeholder"):
#         Z_x = tf.placeholder(tf.float32,[None,m])
#         Z_y = tf.placeholder(tf.float32,[None,n])
        X = tf.placeholder(tf.float32,[None,m])
        Y = tf.placeholder(tf.float32,[None,n])
        
    with tf.variable_scope("cycGAN"):
        false_y = func.G(X,m)
        false_x = func.F(Y,n)
        false_y_x = func.F(false_y,n)
        false_x_y = func.G(false_x,m)
        false_Dy_logit, false_Dy_prob = func.Dy(false_y)
        real_Dy_logit, real_Dy_prob = func.Dy(Y)
        false_Dx_logit, false_Dx_prob = func.Dx(false_x)
        real_Dx_logit, real_Dx_prob = func.Dx(X)
        
    with tf.variable_scope("gan_loss"):
        G_loss = lamda*tf.reduce_mean(tf.square(false_y_x-X))+lamda*tf.reduce_mean(
            tf.square(false_x_y-Y))+tf.reduce_mean(tf.square(false_Dy_prob-1))
        F_loss = lamda*tf.reduce_mean(tf.square(false_y_x-X))+lamda*tf.reduce_mean(
            f.square(false_x_y-Y))+tf.reduce_mean(tf.square(false_Dx_prob-1))
        Dy_loss = tf.reduce_mean(tf.square(real_Dy_prob-1))+tf.reduce_mean(tf.square(false_Dy_prob))
        Dx_loss = tf.reduce_mean(tf.square(real_Dx_prob-1))+tf.reduce_mean(tf.square(false_Dx_prob))
        
    with tf.Variable_scope("train"):
        G_step = tf.train.AdamOptimizer(learning_rate).minimize(G_loss)
        F_step = tf.train.AdamOptimizer(learning_rate).minimize(F_loss)
        Dy_step = tf.train.AdamOptimizer(learning_rate).minimize(Dy_loss)
        Dx_step = tf.train.AdamOptimizer(learning_rate).minimize(Dx_loss)
        
    init_op = tf.global_variables_initializer()
    
    x_batch = get_batch(data["x"])
    y_batch = get_batch(data["y"])
    
    with tf.Session() as sess:
        sess.run(init_op)
        for i in range(1000):
            data_x, data_y = sess.run([x_batch,y_batch])
            _, gloss = sess.run([G_step,G_loss],feed_dict={X:data_x,Y:data_y})
            _, floss = sess.run([F_step,F_loss],feed_dict={X:data_x,Y:data_y})
            _, dyloss = sess.run([Dy_step,Dy_loss],feed_dict={X:data_x,Y:data_y})
            _, dxloss = sess.run([Dx_step,Dx_loss],feed_dict={X:data_x,Y:data_y})
            if i % 10 == 0:
                print("Epoch:{5},Gloss:{1},Floss:{2},Dyloss:{3},Dxloss:{4}".format(gloss,floss,dyloss,dxloss,i))